In [1]:
import numpy as np
import pandas as pd
from numpy.random import choice, randint
import matplotlib.pyplot as plt

import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore

db = firestore.client()

ValueError: The default Firebase app does not exist. Make sure to initialize the SDK by calling initialize_app().

In [ ]:
devs = db.collection('developers')
languages = {
    u"python":50,
    u"node":25,
    u"go":10,
    u"c#":-10,
    u"c++":25,
    u"c":50,
}

for i in range(200):
    dev = {
        "firebase": randint(0,2),
        "language": choice(list(languages.keys())),
        "age": randint(0,99),
    }
    
    r = randint(-25, 25)
    dev['happiness'] = r + languages[dev['language']] + (25 if dev['firebase'] else -25)
    devs.add(dev)

In [ ]:
docs = devs.get()
data = []
for doc in docs:
    data.append(doc.to_dict())
df = pd.DataFrame(data)
df.to_csv('developers.csv', index=False)

In [ ]:
df = pd.read_csv("developers.csv")
df.sample(10)

In [ ]:
df['happiness'].plot.hist()

In [ ]:
df.describe()

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df['language'] = le.fit_transform(df['language'])
le.classes_

In [ ]:
df.sample(10)

In [ ]:
from sklearn.model_selection import train_test_split

x = df.drop(labels='happiness', axis=1)
y = df['happiness']

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=23)
X_train.tail()

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

model = RandomForestRegressor(max_depth=3, random_state=23, n_estimators=500)
model.fit(X_train, y_train)

preds = model.predict(X_test)
print(f"Model Mean Absolute Error; MAE : {mean_absolute_error(y_test, preds)}")

In [ ]:
randos = randint(-100, high=100, size=len(y_test))
print(f"Random Prediction; MAE : {mean_absolute_error(y_test, randos)}")